In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import pickle
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from lesson_functions import *
from scipy.ndimage.measurements import label

In [2]:
# Undistort image
with open('wide_dist_pickle.p', mode='rb') as f:
    calibration_data = pickle.load(f)
    
mtx = calibration_data["mtx"]
dist = calibration_data["dist"]

def undistort(img):
    return cv2.undistort(img, mtx, dist, None, mtx)

In [3]:
# Load SVC data
with open('svc.pickle', mode='rb') as f:
    dict_pickle = pickle.load(f)

svc = dict_pickle["svc"]
X_scaler = dict_pickle["X_scaler"]
color_space = dict_pickle["color_space"]
orient = dict_pickle["orient"]
pix_per_cell = dict_pickle["pix_per_cell"]
cell_per_block = dict_pickle["cell_per_block"]
hog_channel = dict_pickle["hog_channel"]
spatial_size = dict_pickle["spatial_size"]
hist_bins = dict_pickle["hist_bins"]
spatial_feat = dict_pickle["spatial_feat"]
hist_feat = dict_pickle["hist_feat"]
hog_feat = dict_pickle["hog_feat"]

In [4]:
ystart = 400
ystop = 656

# low pass filter version
def process_image(image):
    #image = undistort(image)
    
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    
    bbox_lists = [];
    
    for scale in [1.5, 2.0]:
        # Execute SVC classifier to find car
        bbox_list = find_cars(image, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, return_bbox = True)
        
        # Add heat to each box in box list
        heat = add_heat(heat,bbox_list)
        
    heat_hist[0] = 0.2 * heat + 0.8 * heat_hist[0]
    
    # Apply threshold to help remove false positives
    heat = apply_threshold(heat_hist[0],0.5)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    
    return draw_img

In [5]:
# Process video with heatmap
from moviepy.editor import VideoFileClip
from IPython.display import HTML

heat_hist = [np.zeros((720,1280),dtype=np.float)]

project_output = 'project_video_result2.mp4'
clip1 = VideoFileClip("project_video.mp4")
project_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time project_clip.write_videofile(project_output, audio=False)

[MoviePy] >>>> Building video project_video_result2.mp4
[MoviePy] Writing video project_video_result2.mp4


100%|██████████████████████████████████████████████████████████████████████████▉| 1260/1261 [10:27<00:00,  1.99it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_result2.mp4 

Wall time: 10min 40s
